# Annotate 2D vid

In [38]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import passes dataframe

In [6]:
passes = pd.read_csv('../dashboard/src/stats/all_passe_with_coordinates.csv', index_col=0)
passes.head()

,start_passe,end_passe,x,y,end_x,end_y,player_start,player_end,team_start,team_end,successful
1,63,164,65.692460,31.726149,17.102715,37.858665,"('0', '1')","('1', '9')",0,1,False
5,555,634,21.316270,49.530840,58.898083,54.562640,"('1', '10')","('0', '6')",1,0,False
6,556,634,21.494547,49.634747,58.898083,54.562640,"('1', '10')","('0', '6')",1,0,False
8,777,879,40.987656,43.936720,12.214892,38.809246,"('1', '0')","('1', '10')",1,1,True
9,822,879,29.937870,33.396630,12.214892,38.809246,"('1', '2')","('1', '10')",1,1,True


In [62]:
passe_lines = passes.loc[(passes["start_passe"] < 70) & (passes["end_passe"] > 70), :]
passe_lines.iloc[0].team_start

0

Read video and add annotations.
Create folder with all frame

In [96]:
cap = cv2.VideoCapture('../full_game_2D.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = 0x00000021
print(fps, width, height, fourcc)

# outvid = cv2.VideoWriter('../full_game_2D_passe.mp4', cv2.VideoWriter_fourcc(*'H264'), fps, (width, height), True)

success, frame = cap.read()
frame_nb = 1
while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    
    passe_lines = passes.loc[(passes["start_passe"] < frame_nb) & (passes["end_passe"] > frame_nb), :]
    
    if len(passe_lines) > 0:
        passe_line = passe_lines.iloc[0]
        
        team_color = (255,0,0) if passe_line.team_start == 0 else (0,0,255)
        
        frame = cv2.rectangle(frame,(0,0),(800,30),team_color,-1)
        frame = cv2.putText(frame, 'Passe', (370, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                            (255, 255, 255), 1, cv2.LINE_AA)
        
    cv2.imwrite(f'../vid/frame{str(frame_nb).zfill(6)}.png', frame)
    frame_nb += 1
    # outvid.write(frame)

30.0 800 520 33


convert to video (codec H264) with ffmpeg

In [97]:
!ffmpeg -framerate 30 -pattern_type glob -i '../vid/*.png' -c:v libx264 -pix_fmt yuv420p '../full_game_2D_passe.mp4'

ffmpeg version 5.1.4-0+deb12u1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12 (Debian 12.2.0-14)
  configuration: --prefix=/usr --extra-version=0+deb12u1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librist --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libtwolame --enable-libvidstab --enab